In [5]:
using Plots
include("coevolution_network_base.jl")
using .CoevolutionNetworkBase
using LaTeXStrings

# Parameters
L = 40.0
dx = 0.3
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.0
alpha = 1.0
gamma = 0.0
D = 0.01
Nh = 10^6

# Initialize viral and immune densities
viral_density = [abs(val) <= 0.5 ? 100.0 : 0.0 for val in x]
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population1 = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density)
population2 = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density2, immune_density)

# Create Network instance
migration_matrix = [0.0 0.1; 0.1 0.0] # Define an appropriate migration matrix
network = Network([population1, population2], migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 80.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

Simulation(Network(Population[Population(40.0, 0.3, 3.0, 15, 2.0, 1.0, 0.0, 0.01, 1000000, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], true, 0.0, [-20.0, -19.7, -19.4, -19.1, -18.8, -18.5, -18.2, -17.9, -17.6, -17.3  …  16.9, 17.2, 17.5, 17.8, 18.1, 18.4, 18.7, 19.0, 19.3, 19.6], 133, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), Population(40.0, 0.3, 3.0, 15, 2.0, 1.0, 0.0, 0.01, 1000000, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.

In [ ]:
# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 80.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

In [6]:
using Profile

# Run the simulation
@profile run_simulation!(simulation)

true

In [7]:
Profile.print()


Overhead ╎ [+additional indent] Count File:Line; Function
  ╎577 @Base\client.jl:522; _start()
  ╎ 577 @Base\client.jl:307; exec_options(opts::Base.JLOptions)
  ╎  577 @Base\Base.jl:457; include(mod::Module, _path::Str...
  ╎   577 @Base\loading.jl:1963; _include(mapexpr::Function, mo...
  ╎    577 @Base\loading.jl:1903; include_string(mapexpr::typeof...
  ╎     577 @Base\boot.jl:370; eval
  ╎    ╎ 577 ...otebook\notebook.jl:32; top-level scope
  ╎    ╎  577 ...\serve_notebook.jl:75; kwcall(::NamedTuple{(:crashre...
  ╎    ╎   577 ...serve_notebook.jl:139; serve_notebook(pipename::Str...
  ╎    ╎    577 ...NRPC\src\typed.jl:67; dispatch_msg(x::VSCodeServer...
  ╎    ╎     577 ...serve_notebook.jl:13; notebook_runcell_request(co...
  ╎    ╎    ╎ 577 ...rver\src\repl.jl:274; withpath(f::VSCodeServer.v...
  ╎    ╎    ╎  577 ...erve_notebook.jl:19; (::VSCodeServer.var"#202#2...
  ╎    ╎    ╎   577 ...e\essentials.jl:816; invokelatest
  ╎    ╎    ╎    577 ...e\essentials.jl:819; #invokelate

 8╎    ╎    ╎    ╎     8   ...twork_base.jl:0; cross_reactive_convol...
  ╎    ╎    ╎    ╎     14  ...twork_base.jl:481; cross_reactive_convol...
  ╎    ╎    ╎    ╎    ╎ 4   @Base\int.jl:188; abs
 4╎    ╎    ╎    ╎    ╎  4   @Base\int.jl:142; flipsign
  ╎    ╎    ╎    ╎    ╎ 10  ...promotion.jl:511; min
10╎    ╎    ╎    ╎    ╎  10  @Base\int.jl:83; <
 9╎    ╎    ╎    ╎     528 ...twork_base.jl:483; cross_reactive_convol...
36╎    ╎    ╎    ╎    ╎ 36  @Base\array.jl:969; setindex!
 7╎    ╎    ╎    ╎    ╎ 7   ...ssentials.jl:13; getindex
27╎    ╎    ╎    ╎    ╎ 27  @Base\float.jl:408; +
51╎    ╎    ╎    ╎    ╎ 51  @Base\float.jl:411; /
  ╎    ╎    ╎    ╎    ╎ 25  ...operators.jl:578; *
25╎    ╎    ╎    ╎    ╎  25  @Base\float.jl:410; *
27╎    ╎    ╎    ╎    ╎ 373 ...ecial\exp.jl:325; exp(x::Float64)
  ╎    ╎    ╎    ╎    ╎  20  ...ecial\exp.jl:210; exp_impl
20╎    ╎    ╎    ╎    ╎   20  ...sentials.jl:513; reinterpret
  ╎    ╎    ╎    ╎    ╎  41  ...ecial\exp.jl:211; exp_impl
41╎    ╎   

 1╎    ╎    ╎    ╎    ╎ 1   @Base\array.jl:969; setindex!
  ╎    ╎    ╎    ╎    ╎ 3   ...nericrand.jl:22; rand
 1╎    ╎    ╎    ╎    ╎  3   ...e\poisson.jl:148; rand(rng::Random.Ta...
 2╎    ╎    ╎    ╎    ╎   2   ...s\poisson.jl:22; rand(rng::Random.Ta...
  ╎    ╎    ╎    ╎   2   ...twork_base.jl:375; single_step_evolve_net...
  ╎    ╎    ╎    ╎    2   ...twork_base.jl:509; calculate_migration_e...
  ╎    ╎    ╎    ╎     1   ...\arraymath.jl:8; -(A::Vector{Float64},...
  ╎    ╎    ╎    ╎    ╎ 1   ...broadcast.jl:862; broadcast_preserving...
  ╎    ╎    ╎    ╎    ╎  1   ...broadcast.jl:873; materialize
  ╎    ╎    ╎    ╎    ╎   1   ...roadcast.jl:898; copy
  ╎    ╎    ╎    ╎    ╎    1   ...roadcast.jl:926; copyto!
  ╎    ╎    ╎    ╎    ╎     1   ...roadcast.jl:973; copyto!
  ╎    ╎    ╎    ╎    ╎    ╎ 1   ...simdloop.jl:77; macro expansion
  ╎    ╎    ╎    ╎    ╎    ╎  1   ...oadcast.jl:974; macro expansion
 1╎    ╎    ╎    ╎    ╎    ╎   1   ...\array.jl:969; setindex!
  ╎    ╎    ╎   

In [8]:
using Pkg
Pkg.add("ProfileView")


   Resolving package versions...


   Installed GR_jll ───────────────────── v0.72.8+0


   Installed hicolor_icon_theme_jll ───── v0.17.0+3


   Installed iso_codes_jll ────────────── v4.11.0+0
   Installed Xorg_libXcomposite_jll ───── v0.4.5+4


   Installed DiffEqBase ───────────────── v6.128.3


...

  ✓ BoundaryValueDiffEq

  ✓ OrdinaryDiffEq


  ✓ DelayDiffEq


  ✓ StochasticDiffEq


  ✓ DifferentialEquations
  96 dependencies successfully precompiled in 349 seconds. 226 already precompiled.
  7 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions
  1 dependency had warnings during precompilation:
┌ Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
│  GKS: cairoplugin.dll: can't load library, error 126 (0x7e)
└  


In [11]:
using ProfileView
using Plots
ProfileView.view()


Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint